In [ ]:
# Imports and defaults
import copy
import itertools
from joblib import Parallel, delayed
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import block_diag

import time


mpl.rcParams["figure.figsize"] = [6, 4]

mpl.rcParams["axes.linewidth"] = 0.75
mpl.rcParams["errorbar.capsize"] = 3
mpl.rcParams["figure.facecolor"] = "w"
mpl.rcParams["grid.linewidth"] = 0.75
mpl.rcParams["lines.linewidth"] = 0.75
mpl.rcParams["patch.linewidth"] = 0.75
mpl.rcParams["xtick.major.size"] = 3
mpl.rcParams["ytick.major.size"] = 3

mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
mpl.rcParams["font.size"] = 10
mpl.rcParams["axes.titlesize"] = "medium"

import platform
print("python %s" % platform.python_version())
print("matplotlib %s" % mpl.__version__)

def linestyle2dashes(style):
  if style == "--":
    return (3, 3)
  elif style == ":":
    return (0.5, 2.5)
  else:
    return (None, None)

In [ ]:
# Bandit environments and simulator
#@title Logistic bandit environment
class LogBandit(object):
  """Logistic bandit."""

  def __init__(self, K, contexts, Theta, sigma):    
    self.K = K  # number of arms
    self.contexts = np.copy(contexts)  # [number of contexts] x d feature matrix
    self.num_contexts = self.contexts.shape[0]  # number of contexts
    self.d = self.contexts.shape[1]  # number of features
    self.Theta = np.copy(Theta)  # [number of arms] x d arm parameters
    self.sigma = sigma #reward noise
    self.randomize()

  def sigmoid(self, x):
    y = 1 / (1 + np.exp(- x))
    return y

  def randomize(self):
    # randomly choose one context per arm (does not have to be the same)
    self.ndx = np.random.randint(self.num_contexts, size=self.K)
    self.X = self.contexts[self.ndx, :]

    # mean and stochastic rewards
    self.mut = self.sigmoid((self.X * self.Theta).sum(axis=-1))
    self.rt = (np.random.rand(self.K) < self.mut).astype(float)
    self.best_arm = np.argmax(self.mut)

  def reward(self, arm):
    # instantaneous reward of the arm
    return self.rt[arm]

  def regret(self, arm):
    # instantaneous regret of the arm
    return self.rt[self.best_arm] - self.rt[arm]

  def pregret(self, arm):
    # expected regret of the arm
    return self.mu[self.best_arm] - self.mu[arm]

  def print(self):
    return "Logistic bandit: %d dimensions, %d arms" % (self.d, self.K)


def evaluate_one(Alg, params, env, n, period_size=1):
  """One run of a bandit algorithm."""
  alg = Alg(env, n, params)

  regret = np.zeros(n // period_size)
  for t in range(n):
    # generate state
    env.randomize()

    # take action and update agent
    arm = alg.get_arm(t)
    alg.update(t, arm, env.reward(arm))

    # track performance
    regret_at_t = env.regret(arm)
    regret[t // period_size] += regret_at_t

  return regret, alg

def evaluate(Alg, params, env, n=1000, period_size=1, printout=True):
  """Multiple runs of a bandit algorithm."""
  if printout:
    print("Evaluating %s" % Alg.print(), end="")
  start = time.time()

  num_exps = len(env)
  regret = np.zeros((n // period_size, num_exps))
  alg = num_exps * [None]

  output = Parallel(n_jobs=-1)(delayed(evaluate_one)(Alg, params, env[ex], n, period_size)
    for ex in range(num_exps))
  for ex in range(num_exps):
    regret[:, ex] = output[ex][0]
    alg[ex] = output[ex][1]
  if printout:
    print(" %.1f seconds" % (time.time() - start))

  if printout:
    total_regret = regret.sum(axis=0)
    print("Regret: %.2f +/- %.2f (median: %.2f, max: %.2f, min: %.2f)" %
      (total_regret.mean(), total_regret.std() / np.sqrt(num_exps),
      np.median(total_regret), total_regret.max(), total_regret.min()))

  return regret, alg

In [ ]:
class LogBanditAlg:
  def __init__(self, env, n, params):
    self.env = env  # bandit environment that the agent interacts with
    self.K = self.env.K  # number of arms
    self.d = self.env.d  # number of features
    self.n = n  # horizon
    self.Theta0 = np.copy(env.mar_Theta0)  # prior means of arm parameters
    self.Sigma0 = np.copy(env.mar_Sigma0)  # prior covariance of arm parameters
    self.sigma = self.env.sigma  # reward noise

    self.irls_Theta = np.copy(env.mar_Theta0)
    self.irls_error = 1e-3
    self.irls_num_iter = 1000
    self.batch_size = 30 #self.n
    
    for attr, val in params.items():
      setattr(self, attr, val)

    # sufficient statistics
    self.pulls = np.zeros(self.K, dtype=int) # number of observations for each arm
    
    self.Lambda0 = np.zeros((self.K, self.d, self.d))
    for i in range(self.K):
      self.Lambda0[i, :, :] = np.linalg.inv(self.Sigma0[i, :, :])

    self.G = np.zeros((self.K, self.d, self.d))
    
    self.context_ndx = [[] for i in range(self.K)]
    self.r = [[] for i in range(self.K)]

  def update(self, t, arm, r):    
    self.r[arm].append(r)
    self.pulls[arm] += 1
    self.context_ndx[arm].append(self.env.ndx[arm])
    
    x = self.env.X[arm, :]
    self.G[arm, :, :] += np.outer(x, x) / np.square(self.sigma)      
    
    
  def sigmoid(self, x):
    y = 1 / (1 + np.exp(- x))
    return y

  def solve(self, arm):
    # iterative reweighted least squares for Bayesian logistic regression
    # Sections 4.3.3 and 4.5.1 in Bishop (2006)
    # Pattern Recognition and Machine Learning
    
    small_eye = 1e-5 * np.eye(self.d)
    theta = np.copy(self.irls_Theta[arm,:])
    nbr_obs = self.pulls[arm]
    t = self.r[arm]
    context_ndx = self.context_ndx[arm]

    num_iter = 0
    
    while num_iter < self.irls_num_iter:
        theta_old = np.copy(theta)
        Gram = small_eye
        Phiyt = np.zeros(self.d)

        if nbr_obs <= self.batch_size:
            batch = np.arange(nbr_obs)
        else:
            batch = np.random.choice(nbr_obs, size=self.batch_size)

        for i in batch:
            x = self.env.contexts[context_ndx[i],:]
            y = self.sigmoid((x * theta).sum())
            Gram += y * (1-y) * np.outer(x, x) 
            Phiyt += (y-t[i]) * x

        PhiRz = Gram.dot(theta) - Phiyt
        theta = np.linalg.solve(Gram, PhiRz)
        
        if np.linalg.norm(theta - theta_old) < self.irls_error:
            break;
        num_iter += 1
    
    if num_iter == self.irls_num_iter:
        self.irls_Theta[arm,:] = self.Theta0[arm,:]
    else:
        self.irls_Theta[arm,:] = np.copy(theta)

    return theta, Gram

class LogUCB(LogBanditAlg):
  def __init__(self, env, n, params):
    LogBanditAlg.__init__(self, env, n, params)

    self.cew = self.confidence_ellipsoid_width(n)
    self.inv_Gt = np.zeros((self.K, self.d, self.d))
    
  def confidence_ellipsoid_width(self, t):
    # Section 4.1 in Filippi (2010)
    # Parametric Bandits: The Generalized Linear Case
    delta = 1 / self.n
    c_m = np.amax(np.linalg.norm(self.env.contexts, axis=1))
    c_mu = 0.25 # minimum derivative of the mean function
    k_mu = 0.25
    kappa = np.sqrt(3 + 2 * np.log(1 + 2 * np.square(c_m / self.sigma)))
    R_max = 1.0
    width = (2 * k_mu * kappa * R_max / c_mu) * \
      np.sqrt(2 * self.d * np.log(t) * np.log(2 * self.d * self.n / delta))
    return width

  def get_arm(self, t):
    self.mu = np.zeros(self.K)
    
    if t==0:
        for i in range(self.K):
            Gt = self.Lambda0[i, :, :] + self.G[i, :, :]
            self.inv_Gt[i, :, :] = np.linalg.inv(Gt)
            theta, _ = self.solve(i)
    else:
        Gt = self.Lambda0[self.At, :, :] + self.G[self.At, :, :]
        self.inv_Gt[self.At, :, :] = np.linalg.inv(Gt)
        theta, _ = self.solve(self.At)
                
    for i in range(self.K):
        # UCBs
        theta_hat = self.irls_Theta[i,:]
        inv_Gt = self.inv_Gt[i, :, :]
        self.mu[i] = self.sigmoid(self.env.X[i, :].dot(theta_hat)) + self.cew * \
          np.sqrt((self.env.X[i, :].dot(inv_Gt).dot(self.env.X[i, :])))

    arm = np.argmax(self.mu)
    self.At = arm
    
    return arm

  @staticmethod
  def print():
    return "GLM-UCB"


class UCBLog(LogBanditAlg):
  def __init__(self, env, n, params):
    LogBanditAlg.__init__(self, env, n, params)

    self.cew = self.confidence_ellipsoid_width(n)
    self.inv_Gt = np.zeros((self.K, self.d, self.d))
    
  def confidence_ellipsoid_width(self, t):
    # Theorem 2 in Li (2017)
    # Provably Optimal Algorithms for Generalized Linear Contextual Bandits
    delta = 1 / self.n
    sigma = 0.5
    kappa = 0.25 # minimum derivative of a constrained mean function
    width = (sigma / kappa) * \
      np.sqrt((self.d / 2) * np.log(1 + 2 * self.n / self.d) + \
      np.log(1 / delta))
    return width

  def get_arm(self, t):
    self.mu = np.zeros(self.K)
    
    if t==0:
        for i in range(self.K):
            Gt = self.Lambda0[i, :, :] + self.G[i, :, :]
            self.inv_Gt[i, :, :] = np.linalg.inv(Gt)
            theta, _ = self.solve(i)
    else:
        Gt = self.Lambda0[self.At, :, :] + self.G[self.At, :, :]
        self.inv_Gt[self.At, :, :] = np.linalg.inv(Gt)
        theta, _ = self.solve(self.At)
                
    for i in range(self.K):
        # UCBs
        theta_hat = self.irls_Theta[i,:]
        inv_Gt = self.inv_Gt[i, :, :]
        self.mu[i] = self.sigmoid(self.env.X[i, :].dot(theta_hat)) + self.cew * \
          np.sqrt((self.env.X[i, :].dot(inv_Gt).dot(self.env.X[i, :])))

    arm = np.argmax(self.mu)
    self.At = arm
    
    return arm

  @staticmethod
  def print():
    return "UCB-GLM"



class LogTS(LogBanditAlg):
  def __init__(self, env, n, params):
    LogBanditAlg.__init__(self, env, n, params)
    self.Grams = np.zeros((self.K, self.d, self.d))

  def get_arm(self, t):
    self.mu = np.zeros(self.K)
    
    if t==0:
        for i in range(self.K):
            thetabar, Gram = self.solve(i)
            #Gram_inv = np.linalg.inv(self.Lambda0[self.At, :, :] + Gram)
            #self.inv_Grams[i,:,:] = Gram_inv
            self.Grams[i,:,:] = Gram
    else:
        thetabar, Gram = self.solve(self.At)
        #Gram_inv = np.linalg.inv(self.Lambda0[self.At, :, :] + Gram)
        #self.inv_Grams[self.At,:,:] = Gram_inv
        self.Grams[self.At,:,:] = Gram
        
    for i in range(self.K):
        # posterior sampling
        Sigma_ti = np.linalg.inv(self.Lambda0[i, :, :] + self.Grams[i, :, :])
        mu_ti = self.Lambda0[i, :, :].dot(self.Theta0[i, :]) + self.Grams[i, :, :].dot(self.irls_Theta[i,:])
        mu_ti = Sigma_ti.dot(mu_ti)
        theta_tilde = np.random.multivariate_normal(mu_ti, Sigma_ti)
        self.mu[i] = self.env.X[i, :].dot(theta_tilde)
    arm = np.argmax(self.mu)
    self.At = arm
    #print(self.irls_Theta[arm, :], arm)
    return arm

  @staticmethod
  def print():
    return "GLM-TSL"

In [ ]:
class meTS:
  def __init__(self, env, n, params):
    self.env = env  # bandit environment that the agent interacts with
    self.K = self.env.K  # number of arms
    self.d = self.env.d  # number of features
    self.n = n  # horizon
    self.A = self.env.A
    self.L = self.A.shape[1]
    self.mu_psi = np.copy(self.env.mu_psi)
    self.Sigma_psi = np.copy(self.env.Sigma_psi)
    self.Sigma0 = np.copy(self.env.Sigma0)
    self.sigma = self.env.sigma  # reward noise
    
    self.irls_Theta = np.zeros((self.K, self.d))
    self.irls_error = 1e-3
    self.irls_num_iter = 1000
    self.batch_size = 30 #self.n
    
    # override default values
    for attr, val in params.items():
      if isinstance(val, np.ndarray):
        setattr(self, attr, np.copy(val))
      else:
        setattr(self, attr, val)

    self.Lambda_psi = np.linalg.inv(self.Sigma_psi)
    self.Lambda0 = np.zeros((self.K, self.d, self.d))
    for i in range(self.K):
      self.Lambda0[i, :, :] = np.linalg.inv(self.Sigma0[i, :, :])
    
    self.Theta0 = np.copy(self.env.mar_Theta0) #np.zeros((self.K, self.d))
    
    # sufficient statistics
    self.G = np.zeros((self.K, self.d, self.d))
    self.B = np.zeros((self.K, self.d))
    
    self.pulls = np.zeros(self.K, dtype=int) # number of observations for each arm    
    self.context_ndx = [[] for i in range(self.K)]
    self.r = [[] for i in range(self.K)]
        
  def sigmoid(self, x):
    y = 1 / (1 + np.exp(- x))
    return y

  def solve(self, arm):
    # iterative reweighted least squares for Bayesian logistic regression
    # Sections 4.3.3 and 4.5.1 in Bishop (2006)
    # Pattern Recognition and Machine Learning
    small_eye = 1e-5 * np.eye(self.d)
    theta = np.copy(self.irls_Theta[arm,:])
    nbr_obs = self.pulls[arm]
    t = self.r[arm]
    context_ndx = self.context_ndx[arm]
    num_iter = 0
    
    while num_iter < self.irls_num_iter:
        theta_old = np.copy(theta)
        Gram =  small_eye #self.Lambda0[arm, :, :]
        Phiyt = np.zeros(self.d)

        if nbr_obs <= self.batch_size:
            batch = np.arange(nbr_obs)
        else:
            batch = np.random.choice(nbr_obs, size=self.batch_size)

        for i in batch:
            x = self.env.contexts[context_ndx[i],:]
            y = self.sigmoid((x * theta).sum())
            Gram += y * (1-y) * np.outer(x, x) 
            Phiyt += (y-t[i]) * x

        PhiRz = Gram.dot(theta) - Phiyt
        theta = np.linalg.solve(Gram, PhiRz)
        
        if np.linalg.norm(theta - theta_old) < self.irls_error:
            break;
        num_iter += 1
    
    if num_iter == self.irls_num_iter:
        self.irls_Theta[arm,:] = self.Theta0[arm,:]
    else:
        self.irls_Theta[arm,:] = np.copy(theta)

    return theta, Gram


  def update(self, t, arm, r):    
    self.r[arm].append(r)
    self.pulls[arm] += 1
    self.context_ndx[arm].append(self.env.ndx[arm])
    
    x = self.env.X[arm, :]
    theta, self.G[arm, :, :] = self.solve(arm)
    self.B[arm,:] = self.G[arm, :, :].dot(theta)

  def get_arm(self, t):
    small_eye = 1e-5 * np.eye(self.d)
    
    # effect parameter posterior
    Lambda_t = np.copy(self.Lambda_psi)
    mu_t = self.Lambda_psi.dot(self.mu_psi)
    for i in range(self.K):
      aiai = np.outer(self.A[i, :], self.A[i, :])
      inv_Gi = np.linalg.inv(self.G[i, :, :] + small_eye)
      prior_adjusted_Gi = np.linalg.inv(self.Sigma0[i, :, :] + inv_Gi)
      Lambda_t += np.kron(aiai, prior_adjusted_Gi)
      prior_adjusted_Bi = prior_adjusted_Gi.dot(inv_Gi.dot(self.B[i, :]))
      mu_t += np.outer(self.A[i, :], prior_adjusted_Bi).flatten()
    
    # posterior sampling
    Sigma_t = np.linalg.inv(Lambda_t)
    mu_t = Sigma_t.dot(mu_t)
    Psi_tilde = np.random.multivariate_normal(mu_t, Sigma_t)
    matPsi_tilde = np.reshape(Psi_tilde, (self.L, self.d))  # matrix version
    
    self.mu = np.zeros(self.K)
    for i in range(self.K):
      # arm parameter posterior
      Sigma_ti = np.linalg.inv(self.Lambda0[i, :, :] + self.G[i, :, :])
      self.Theta0[i, :] = self.A[i, :].dot(matPsi_tilde)
      mu_ti = self.Lambda0[i, :, :].dot(self.Theta0[i, :]) + self.B[i, :]
      mu_ti = Sigma_ti.dot(mu_ti)
      
      # posterior sampling
      theta_tilde = np.random.multivariate_normal(mu_ti, Sigma_ti)
      self.mu[i] = self.env.X[i, :].dot(theta_tilde)
    
    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "meTS-GLM"

In [ ]:
class FactoredmeTS:
  def __init__(self, env, n, params):
    self.env = env  # bandit environment that the agent interacts with
    self.K = self.env.K  # number of arms
    self.d = self.env.d  # number of features
    self.n = n  # horizon
    self.A = self.env.A
    self.L = self.A.shape[1]
    self.mu_psi = np.copy(self.env.mu_psi)
    self.Sigma_psi = np.copy(self.env.Sigma_psi)
    self.Sigma0 = np.copy(self.env.Sigma0)
    self.sigma = self.env.sigma  # reward noise
    
    self.irls_Theta = np.zeros((self.K, self.d))
    self.irls_error = 1e-3
    self.irls_num_iter = 1000
    self.batch_size = 30 #self.n
    
    # override default values
    for attr, val in params.items():
      if isinstance(val, np.ndarray):
        setattr(self, attr, np.copy(val))
      else:
        setattr(self, attr, val)

    self.Lambda_psi = np.linalg.inv(self.Sigma_psi)
    self.Lambda0 = np.zeros((self.K, self.d, self.d))
    for i in range(self.K):
      self.Lambda0[i, :, :] = np.linalg.inv(self.Sigma0[i, :, :])
    
    self.Theta0 = np.copy(self.env.mar_Theta0) #np.zeros((self.K, self.d))
    
    # sufficient statistics
    self.G = np.zeros((self.K, self.d, self.d))
    self.B = np.zeros((self.K, self.d))
    
    self.pulls = np.zeros(self.K, dtype=int) # number of observations for each arm    
    self.context_ndx = [[] for i in range(self.K)]
    self.r = [[] for i in range(self.K)]
        
  def sigmoid(self, x):
    y = 1 / (1 + np.exp(- x))
    return y

  def solve(self, arm):
    # iterative reweighted least squares for Bayesian logistic regression
    # Sections 4.3.3 and 4.5.1 in Bishop (2006)
    # Pattern Recognition and Machine Learning
    small_eye = 1e-5 * np.eye(self.d)
    theta = np.copy(self.irls_Theta[arm,:])
    nbr_obs = self.pulls[arm]
    t = self.r[arm]
    context_ndx = self.context_ndx[arm]
    num_iter = 0
    
    while num_iter < self.irls_num_iter:
        theta_old = np.copy(theta)
        Gram =  small_eye #self.Lambda0[arm, :, :]
        Phiyt = np.zeros(self.d)

        if nbr_obs <= self.batch_size:
            batch = np.arange(nbr_obs)
        else:
            batch = np.random.choice(nbr_obs, size=self.batch_size)

        for i in batch:
            x = self.env.contexts[context_ndx[i],:]
            y = self.sigmoid((x * theta).sum())
            Gram += y * (1-y) * np.outer(x, x) 
            Phiyt += (y-t[i]) * x

        PhiRz = Gram.dot(theta) - Phiyt
        theta = np.linalg.solve(Gram, PhiRz)
        
        if np.linalg.norm(theta - theta_old) < self.irls_error:
            break;
        num_iter += 1
    
    if num_iter == self.irls_num_iter:
        self.irls_Theta[arm,:] = self.Theta0[arm,:]
    else:
        self.irls_Theta[arm,:] = np.copy(theta)

    return theta, Gram


  def update(self, t, arm, r):    
    self.r[arm].append(r)
    self.pulls[arm] += 1
    self.context_ndx[arm].append(self.env.ndx[arm])
    
    x = self.env.X[arm, :]
    theta, self.G[arm, :, :] = self.solve(arm)
    self.B[arm,:] = self.G[arm, :, :].dot(theta)

  def get_arm(self, t):
    small_eye = 1e-5 * np.eye(self.d)
    
    # effect parameter posterior
    Lambda_t = []
    mu_t = []
    for l in range(self.L):
      Lambda_psi_l = self.Lambda_psi[l*self.d:(l+1)*self.d,l*self.d:(l+1)*self.d]
      Lambda_t.append(np.copy(Lambda_psi_l))
      mu_t.append(Lambda_psi_l.dot(self.mu_psi[l*self.d:(l+1)*self.d]))

    for i in range(self.K):
      inv_Gi = np.linalg.inv(self.G[i, :, :] + small_eye)
      prior_adjusted_Gi = np.linalg.inv(self.Sigma0[i, :, :] + inv_Gi)
      prior_adjusted_Bi = prior_adjusted_Gi.dot(inv_Gi.dot(self.B[i, :]))
      for l in range(self.L):
        Lambda_t[l] += (self.A[i, l]**2) * prior_adjusted_Gi
        mu_t[l] += self.A[i, l] * prior_adjusted_Bi
    
    # posterior sampling
    Sigma_t = []
    for l in range(self.L):
      Sigma_t.append(np.linalg.inv(Lambda_t[l]))
      mu_t[l] = Sigma_t[l].dot(mu_t[l])

    Sigma_t = block_diag(*Sigma_t)
    mu_t = np.concatenate(mu_t)
    
    Psi_tilde = np.random.multivariate_normal(mu_t, Sigma_t)
    matPsi_tilde = np.reshape(Psi_tilde, (self.L, self.d))  # matrix version
    
    self.mu = np.zeros(self.K)
    for i in range(self.K):
      # arm parameter posterior
      Sigma_ti = np.linalg.inv(self.Lambda0[i, :, :] + self.G[i, :, :])
      self.Theta0[i, :] = self.A[i, :].dot(matPsi_tilde)
      mu_ti = self.Lambda0[i, :, :].dot(self.Theta0[i, :]) + self.B[i, :]
      mu_ti = Sigma_ti.dot(mu_ti)
      
      # posterior sampling
      theta_tilde = np.random.multivariate_normal(mu_ti, Sigma_ti)
      self.mu[i] = self.env.X[i, :].dot(theta_tilde)
    
    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "meTS-GLM-Fa"

In [ ]:
class LinmeTS:
  def __init__(self, env, n, params):
    self.env = env  # bandit environment that the agent interacts with
    self.K = self.env.K  # number of arms
    self.d = self.env.d  # number of features
    self.n = n  # horizon
    self.A = self.env.A
    self.L = self.A.shape[1]
    self.mu_psi = np.copy(self.env.mu_psi)
    self.Sigma_psi = np.copy(self.env.Sigma_psi)
    self.Sigma0 = np.copy(self.env.Sigma0)
    self.sigma = self.env.sigma  # reward noise

    # override default values
    for attr, val in params.items():
      if isinstance(val, np.ndarray):
        setattr(self, attr, np.copy(val))
      else:
        setattr(self, attr, val)

    self.Lambda_psi = np.linalg.inv(self.Sigma_psi)
    self.Lambda0 = np.zeros((self.K, self.d, self.d))
    for i in range(self.K):
      self.Lambda0[i, :, :] = np.linalg.inv(self.Sigma0[i, :, :])

    # sufficient statistics
    self.G = np.zeros((self.K, self.d, self.d))
    self.B = np.zeros((self.K, self.d))

  def update(self, t, arm, r):
    # update sufficient statistics
    x = self.env.X[arm, :]
    self.G[arm, :, :] += np.outer(x, x) / np.square(self.sigma)
    self.B[arm, :] += x * r / np.square(self.sigma)

  def get_arm(self, t):
    small_eye = 1e-3 * np.eye(self.d)
    
    # effect parameter posterior
    Lambda_t = np.copy(self.Lambda_psi)
    mu_t = self.Lambda_psi.dot(self.mu_psi)
    for i in range(self.K):
      aiai = np.outer(self.A[i, :], self.A[i, :])
      inv_Gi = np.linalg.inv(self.G[i, :, :] + small_eye)
      prior_adjusted_Gi = np.linalg.inv(self.Sigma0[i, :, :] + inv_Gi)
      Lambda_t += np.kron(aiai, prior_adjusted_Gi)
      prior_adjusted_Bi = prior_adjusted_Gi.dot(inv_Gi.dot(self.B[i, :]))
      mu_t += np.outer(self.A[i, :], prior_adjusted_Bi).flatten()
    
    # posterior sampling
    Sigma_t = np.linalg.inv(Lambda_t)
    mu_t = Sigma_t.dot(mu_t)
    Psi_tilde = np.random.multivariate_normal(mu_t, Sigma_t)
    matPsi_tilde = np.reshape(Psi_tilde, (self.L, self.d))  # matrix version
    
    self.mu = np.zeros(self.K)
    for i in range(self.K):
      # arm parameter posterior
      Sigma_ti = np.linalg.inv(self.Lambda0[i, :, :] + self.G[i, :, :])
      mu_ti = self.Lambda0[i, :, :].dot(self.A[i, :].dot(matPsi_tilde)) + self.B[i, :]
      mu_ti = Sigma_ti.dot(mu_ti)
      
      # posterior sampling
      theta_tilde = np.random.multivariate_normal(mu_ti, Sigma_ti)
      self.mu[i] = self.env.X[i, :].dot(theta_tilde)
    
    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "meTS-Lin"

In [ ]:
class HierTS:
  def __init__(self, env, n, params):
    self.env = env  # bandit environment that the agent interacts with
    self.K = self.env.K  # number of arms
    self.d = self.env.d  # number of features
    self.n = n  # horizon
    self.A = self.env.A
    self.L = self.A.shape[1]
    self.mu_psi = np.mean(self.env.mu_psi.reshape(self.L, self.d), axis=0) # average of effect vectors as a single d-dimensional effect vector
    self.Sigma_psi = (1/(self.L**2)) * np.sum([self.env.Sigma_psi[l*self.d:(l+1)*self.d,l*self.d:(l+1)*self.d] for l in range(self.L)], axis=0) # corresponding cov matrix
    self.Sigma0 = np.copy(self.env.Sigma0)
    self.sigma = self.env.sigma  # reward noise

    # override default values
    for attr, val in params.items():
      if isinstance(val, np.ndarray):
        setattr(self, attr, np.copy(val))
      else:
        setattr(self, attr, val)

    self.Lambda_psi = np.linalg.inv(self.Sigma_psi)
    self.Lambda0 = np.zeros((self.K, self.d, self.d))
    for i in range(self.K):
      self.Lambda0[i, :, :] = np.linalg.inv(self.Sigma0[i, :, :])

    # sufficient statistics
    self.G = np.zeros((self.K, self.d, self.d))
    self.B = np.zeros((self.K, self.d))

  def update(self, t, arm, r):
    # update sufficient statistics
    x = self.env.X[arm, :]
    self.G[arm, :, :] += np.outer(x, x) / np.square(self.sigma)
    self.B[arm, :] += x * r / np.square(self.sigma)

  def get_arm(self, t):
    small_eye = 1e-3 * np.eye(self.d)
    
    # effect parameter posterior
    Lambda_t = np.copy(self.Lambda_psi)
    mu_t = self.Lambda_psi.dot(self.mu_psi)
    for i in range(self.K):
      inv_Gi = np.linalg.inv(self.G[i, :, :] + small_eye)
      prior_adjusted_Gi = np.linalg.inv(self.Sigma0[i, :, :] + inv_Gi)
      Lambda_t += prior_adjusted_Gi
      prior_adjusted_Bi = prior_adjusted_Gi.dot(inv_Gi.dot(self.B[i, :]))
      mu_t += prior_adjusted_Bi
    
    # posterior sampling
    Sigma_t = np.linalg.inv(Lambda_t)
    mu_t = Sigma_t.dot(mu_t)
    Psi_tilde = np.random.multivariate_normal(mu_t, Sigma_t)
    
    self.mu = np.zeros(self.K)
    for i in range(self.K):
      # arm parameter posterior
      Sigma_ti = np.linalg.inv(self.Lambda0[i, :, :] + self.G[i, :, :])
      mu_ti = self.Lambda0[i, :, :].dot(Psi_tilde) + self.B[i, :]
      mu_ti = Sigma_ti.dot(mu_ti)
      
      # posterior sampling
      theta_tilde = np.random.multivariate_normal(mu_ti, Sigma_ti)
      self.mu[i] = self.env.X[i, :].dot(theta_tilde)
    
    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "HierTS"

In [ ]:
sigma = 1.0
n = 5000
num_runs = 50

algs = [
  ("meTS", {}, "red", "-", "meTS-GLM"),
  ("FactoredmeTS", {}, "orange", "-", "meTS-GLM-Fa"),
  ("LinmeTS", {}, "purple", "-", "meTS-Lin"),
 ("LogTS", {}, "blue", "-", "GLM-TS"),
  ("UCBLog", {}, "cyan", "-", "UCB-GLM"),
   ("HierTS", {}, "gray", "-", "HierTS"),
]


#exps = [
#  {"K": 100, "L": 3, "d": 2},
#  {"K": 100, "L": 3, "d": 5},
#  {"K": 100, "L": 3, "d": 10},
#  {"K": 50, "L": 3, "d": 2},
#  {"K": 50, "L": 3, "d": 5},
#  {"K": 50, "L": 3, "d": 10},
#  {"K": 20, "L": 3, "d": 2},
#  {"K": 20, "L": 3, "d": 5},
#  {"K": 20, "L": 3, "d": 10}
#]

exps = [
  {"K": 100, "L": 3, "d": 2}
]


for exp in exps:
  # set parameters of the experiment
  for attr, val in exp.items():
    globals()[attr] = val

  # bandit environments
  envs = []
  for run in range(num_runs):
    # possible contexts
    contexts = 2 * np.random.rand(100, d) - 1

    # mixing coefficients
    A = 3 * np.random.rand(K, L) - 1
    
    #matA: matrix formulation of the mixing weights for action parameters  
    matA = np.zeros((K*d, L*d))
    for i in range(K):
        matA[i*d:(i+1)*d, :] = np.kron(A[i, :].T, np.eye(d))

    # effect and arm priors
    mu_psi = np.zeros(d * L)
    mat_mu_Psi = np.reshape(mu_psi, (L, d))
    Sigma_psi = 3 * np.eye(d * L)
    Sigma0 = np.tile(np.eye(d), (K, 1, 1))
    
    # marginal uncertainty of arm parameters
    mar_Theta0 = np.zeros((K, d))
    mar_Sigma0 = np.copy(Sigma0)        
    for i in range(K):
        matAi = matA[i*d:(i+1)*d, :]
        mar_Theta0[i, :] = matAi.dot(mu_psi)
        mar_Sigma0[i, :, :] = Sigma0[i, :, :] + matAi.dot(Sigma_psi).dot(matAi.T)

    # generate effect parameters
    Psi = np.random.multivariate_normal(mu_psi, Sigma_psi)
    matPsi = np.reshape(Psi, (L, d))  # matrix version

    # generate arm parameters
    Theta = np.random.randn(K, d)
    for i in range(K):
      Theta[i, :] = np.random.multivariate_normal(A[i, :].dot(matPsi), Sigma0[i, :, :])
    # initialize bandit environment
    env = LogBandit(K, contexts, Theta, sigma=sigma)

    # pass parameters for algorithm initialization (not used in simulation)
    env.A = A
    env.mu_psi = mu_psi
    env.Sigma_psi = Sigma_psi
    env.Sigma0 = Sigma0
    env.mar_Theta0 = mar_Theta0
    env.mar_Sigma0 = mar_Sigma0

    envs.append(env)
    
  #trained_algs = []
  # simulation
  for alg in algs:
    # all runs for a single algorithm
    alg_class = globals()[alg[0]]
    regret, logs = evaluate(alg_class, alg[1], envs, n)
    #regret, logs = evaluate_one(alg_class, alg[1], envs[0], n)
    
    #trained_algs.append(logs)
    # # save results
    #fname = "Results/log_%s_K=%d_L=%d_d=%d" % (alg[4], K, L, d)
    #np.save(fname + ".npy", regret)

    # plot
    cum_regret = regret.cumsum(axis=0)
    step = np.arange(1, n + 1)
    sube = (step.size // 10) * np.arange(1, 11) - 1
    plt.plot(step, cum_regret.mean(axis=1),
      alg[2], dashes=linestyle2dashes(alg[3]), label=alg[4])
    plt.errorbar(step[sube], cum_regret[sube, :].mean(axis=1),
      cum_regret[sube, :].std(axis=1) / np.sqrt(cum_regret.shape[1]),
      fmt="none", ecolor=alg[2])

  plt.title("Log Bandit, K = %d, L = %d, d = %d" % (K, L, d))
  plt.xlabel("Round n")
  plt.ylabel("Regret")
  plt.ylim(bottom=0)
  plt.legend(loc="upper left", frameon=False)

  plt.tight_layout()
  plt.show()